In [1]:
import numpy as np
import pandas as pd
import os
import csv
import zipfile
import re

In [2]:
output_dir = os.listdir(os.path.join(os.getcwd(),'Teradata_Outputs'))

In [3]:
allrights = pd.read_csv(os.path.join(os.getcwd(),'Teradata_Outputs',output_dir[0]))
allrolerights = pd.read_csv(os.path.join(os.getcwd(),'Teradata_Outputs',output_dir[1]))
profiles = pd.read_excel(os.path.join(os.getcwd(),'Teradata_Outputs',output_dir[2]), engine='openpyxl')
users = pd.read_csv(os.path.join(os.getcwd(),'Teradata_Outputs',output_dir[3]))
teradata_tc = pd.read_excel('Teradata_TC.xlsx', engine='openpyxl')
nomina = pd.read_excel('nomina.xlsx', engine='openpyxl')
nomina['Nombre de usuario'] =  nomina['Nombre de usuario'].apply(lambda x: str(x).upper()).apply(lambda x: x.replace(" ",""))
nomina_externos = pd.read_excel('nomina_externos.xlsx', engine='openpyxl')
nomina_externos['NOMBRE_USUARIO'] = nomina_externos['NOMBRE_USUARIO'].apply(lambda x: str(x).upper()).apply(lambda x: x.replace(" ",""))
nomina_externos.rename(columns={'DEPARTAMENTO':'Departamento_ext'}, inplace=True)

In [4]:
allrights = allrights.applymap(lambda x: str(x).upper())
allrights[['DatabaseName','TableName','AccessRight']] = allrights[['DatabaseName','TableName','AccessRight']].applymap(lambda x: x.replace(" ",""))
allrolerights = allrolerights.applymap(lambda x: str(x).upper())
allrolerights[['RoleName','DatabaseName','TableName','AccessRight']] = allrolerights[['RoleName','DatabaseName','TableName','AccessRight']].applymap(lambda x: str(x).upper()).applymap(lambda x: x.replace(" ",""))
profiles[['ProfileNameI','ProfileName']] =  profiles[['ProfileNameI','ProfileName']].applymap(lambda x: str(x).upper()).applymap(lambda x: x.replace(" ",""))
users[['USERNAME','CreatorName','OwnerName','RoleName','ProfileName']] = users[['USERNAME','CreatorName','OwnerName','RoleName','ProfileName']].applymap(lambda x: str(x).upper()).applymap(lambda x: x.replace(" ",""))
teradata_tc = teradata_tc.applymap(lambda x: str(x).upper())

In [5]:
allrights = allrights[allrights['UserName'] != allrights['DatabaseName']].reset_index(drop=True)

In [6]:
mask_relevant_allrights = ((allrights['DatabaseName'] == 'DBC') | (allrights['DatabaseName'] == 'WHOWNER') | \
                          (allrights['DatabaseName'] == 'QUALES_PROD') | (allrights['DatabaseName'] == 'MSTR_TODATE')| \
                          #(allrights['DatabaseName'] == 'TEMP_45') | \
                          (allrights['DatabaseName'] == 'DW_WHOWNER_TBL')) &  ((allrights['AccessRight'] == 'D') | \
                            (allrights['AccessRight'] == 'U') | (allrights['AccessRight'] == 'DA') | \
                            (allrights['AccessRight'] == 'CA') | (allrights['AccessRight'] == 'I') | \
                            (allrights['AccessRight'] == 'DT') | (allrights['AccessRight'] == 'DU') | \
                            (allrights['AccessRight'] == 'CR') | (allrights['AccessRight'] == 'DR') | \
                            (allrights['AccessRight'] == 'DS') | (allrights['AccessRight'] == 'DD'))

mask_relevant_allrolerights = ((allrolerights['DatabaseName'] == 'DBC')   | (allrolerights['DatabaseName'] == 'WHOWNER') | \
                              (allrolerights['DatabaseName'] == 'QUALES_PROD') | (allrolerights['DatabaseName'] == 'MSTR_TODATE')| \
                              #(allrolerights['DatabaseName'] == 'TEMP_45') | \
                              (allrolerights['DatabaseName'] == 'DW_WHOWNER_TBL')) &  ((allrolerights['AccessRight'] == 'D') | \
                                (allrolerights['AccessRight'] == 'U') | (allrolerights['AccessRight'] == 'DA') | \
                                (allrolerights['AccessRight'] == 'CA') | (allrolerights['AccessRight'] == 'I') | \
                                (allrolerights['AccessRight'] == 'DT') | (allrolerights['AccessRight'] == 'DU') | \
                                (allrolerights['AccessRight'] == 'CR') | (allrolerights['AccessRight'] == 'DR') | \
                                (allrolerights['AccessRight'] == 'DS') | (allrolerights['AccessRight'] == 'DD'))
                            

In [7]:
allrights = allrights[mask_relevant_allrights].reset_index(drop=True)
allrolerights = allrolerights[mask_relevant_allrolerights].reset_index(drop=True)

In [8]:
allrights = allrights.merge(teradata_tc['TABLE_NAME'], how='left', left_on='TableName', right_on='TABLE_NAME')
allrights.rename(columns={'TABLE_NAME':'Tabla_Critica'}, inplace=True)
allrolerights = allrolerights.merge(teradata_tc['TABLE_NAME'], how='left', left_on='TableName', right_on='TABLE_NAME')
allrolerights.rename(columns={'TABLE_NAME':'Tabla_Critica'}, inplace=True)

In [9]:
manual_tc = ['ALL', 'USERS', 'ALLRIGHTS', 'PROFILES']
c1 = allrights['TableName'].str.match('|'.join(manual_tc))
allrights['Tabla_Critica'][allrights['Tabla_Critica'].isnull()] = np.select([c1], [allrights['Tabla_Critica'].fillna(1)])
allrights['Tabla_Critica'].loc[~allrights['Tabla_Critica'].isnull()] = 1
allrights['Tabla_Critica'].loc[allrights['Tabla_Critica'].isnull()] = 0
c2 = allrolerights['TableName'].str.match('|'.join(manual_tc))
allrolerights['Tabla_Critica'][allrolerights['Tabla_Critica'].isnull()] = np.select([c2], [allrolerights['Tabla_Critica'].fillna(1)])
allrolerights['Tabla_Critica'].loc[~allrolerights['Tabla_Critica'].isnull()] = 1
allrolerights['Tabla_Critica'].loc[allrolerights['Tabla_Critica'].isnull()] = 0

In [10]:
allrights['UserName_1'] = allrights['UserName'].apply(lambda x: x.replace("_DBA", ""))
allrights['UserName_1'] = allrights['UserName_1'].apply(lambda x: x.replace("MS_", ""))

In [11]:
allrights = allrights.merge(nomina[['Nombre de usuario', 'Departamento']], how='left', left_on='UserName_1', right_on='Nombre de usuario').drop('Nombre de usuario', axis=1)
allrights = allrights.merge(nomina_externos[['NOMBRE_USUARIO', 'Departamento_ext']], how='left', left_on='UserName_1', right_on='NOMBRE_USUARIO').drop('NOMBRE_USUARIO', axis=1)

In [12]:
ad_users = pd.read_csv('AD.csv',sep=',',low_memory=False)
ad_users = ad_users[['Name', 'Disabled']].applymap(lambda x: str(x).upper()).applymap(lambda x: x.replace(" ",""))
ad_users.rename(columns={'Disabled':'AD_Disabled'}, inplace=True)

In [13]:
allrights = allrights.merge(ad_users, how='left', left_on='UserName_1', right_on='Name').drop('Name', axis=1)


In [14]:
app_users = ['SYSLIB', 'FRAUD', 'APP_QUALES_PROD',
       'SELL_TERADATA', 'ODS', 'APP_QUALES', 'APP_CDR_USER_BEHAVIORS',
       'APP_PENTAHO_ETL', 'MSTRADMIN', 'APP_CBK_ANALYSIS', 'DBCMANAGER',
       'APP_MERCHXPS', 'APP_GAIA_01', 'TABLEAU', 'TABLEAU_TBL',
       'VIEWPOINT', 'APP_PENTAHO', 'MSTRADMIN_H',
       'APP_BIWEB', 'APP_CREDITS_BACKEND', 'TDPUSER', 'SYSDBA',
       'APP_CUST_FUNDATION', 'APP_CCNMP_PROD', 'MODELING',
       'APP_OPERACIONESMP', 'APP_LOGISTICS_ENVIOS', 'SYSBACK',
       'APP_MELI_LOADER', 'CONNECT_HADOOP', 'QUALES_PROD', 'APP_GAIA',
       'APP_ANALYTICS', 'APP_MLD_FORECASTING', 'MELI', 'APP_ETL',
       'ODS_TABLEAU', 'APPMIGRADORDESA', 'APP_STATS', 'BI', 'SYSTEMFE',
       'APP_GENOVA', 'RESTOREAR', 'MELI_LIB',
       'APP_FRAUD_MEJORA_CONTINUA', 'APP_CDG', 'MP3_WP', 'DBCMNGR',
       'APP_QUALES_TEMP', 'APPMIGRADORFE', 'NWC', 'BACKUP', 'WH_TAXREP',
       'SYSADMIN', 'APP_SM_MS', 'BARUSER1', 'APP_PYME_CBA_TERADATA',
       'TABLEAU_RT', 'APP_LAG', 'SP_BI', 'ANALYTICS_DATA']
schema_users = ['DBC', """"'TEMP_45'""", 'DW_WHOWNER_TBL', 'WHOWNER_TBL', 'MSTR_TODATE']
public_users = ['PUBLIC']
c3 = allrights['UserName_1'].str.match('|'.join(app_users))
c4 = allrights['UserName_1'].str.match('|'.join(schema_users))
c5 = allrights['UserName_1'].str.match('|'.join(public_users))
allrights['Departamento'][allrights['Departamento'].isnull()] = np.select([c3,c4,c5], 
                                                                          [allrights['Departamento'].fillna('Usuario de aplicativo'),
                                                                          allrights['Departamento'].fillna('Usuario de schema'),
                                                                          allrights['Departamento'].fillna('Permisos publicos')])

In [15]:
allrolerights = allrolerights[allrolerights['Tabla_Critica']==1]
allrolerights[['RoleName', 'DatabaseName']].to_csv('relevant_roles.csv')
#allrolerights = allrolerights[['RoleName', 'DatabaseName']].drop_duplicates().reset_index(drop=True)

In [16]:
allrolerights.columns

Index(['RoleName', 'DatabaseName', 'TableName', 'ColumnName', 'AccessRight',
       'GrantorName', 'CreateTimeStamp', 'Tabla_Critica'],
      dtype='object')

In [17]:
allrights.columns

Index(['UserName', 'DatabaseName', 'TableName', 'AccessRight', 'Tabla_Critica',
       'UserName_1', 'Departamento', 'Departamento_ext', 'AD_Disabled'],
      dtype='object')

In [18]:
users.columns

Index(['USERNAME', 'CreatorName', 'PasswordLastModDate', 'PasswordLastModTime',
       'OwnerName', 'DefaultDataBase', 'CommentString', 'LockedDate',
       'LockedTime', 'LockedCount', 'RoleName', 'ProfileName',
       'PasswordChgDate', 'CreateTimeStamp', 'LastAlterName',
       'LastAlterTimeStamp'],
      dtype='object')

In [19]:
allrolerights = allrolerights.merge(users[['USERNAME','RoleName']], how='left', on='RoleName', validate='many_to_many').drop_duplicates().reset_index(drop=True)

In [20]:
allrolerights.columns

Index(['RoleName', 'DatabaseName', 'TableName', 'ColumnName', 'AccessRight',
       'GrantorName', 'CreateTimeStamp', 'Tabla_Critica', 'USERNAME'],
      dtype='object')

In [21]:
allrolerights = allrolerights.merge(allrights[['UserName','UserName_1','Departamento','Departamento_ext', 'AD_Disabled']], how='left', left_on='USERNAME', right_on='UserName').drop_duplicates().reset_index(drop=True)

In [22]:
allrolerights['Comentario'] = np.nan
allrolerights['Comentario'][allrolerights['UserName'].isnull()] = 'Rol no asignado a usuarios'

<ipython-input-22-6316481d4ce6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allrolerights['Comentario'][allrolerights['UserName'].isnull()] = 'Rol no asignado a usuarios'


In [23]:
allrolerights.to_csv('roles-users.csv')

In [24]:
relevant_users = allrights['UserName'].append(allrolerights['UserName']).drop_duplicates().reset_index(drop=True)
relevant_users = pd.DataFrame(relevant_users)

In [25]:
relevant_users.columns

Index(['UserName'], dtype='object')

In [26]:
relevant_profiles = relevant_users.merge(users[['USERNAME','ProfileName']], how='left', left_on='UserName', right_on='USERNAME').dropna(how='all')

In [27]:
relevant_profiles.to_csv('users-profiles_relevant.csv')

In [28]:
relevant_profiles = relevant_profiles['ProfileName'].dropna().drop_duplicates().reset_index(drop=True)
relevant_profiles.to_csv('relevant_profiles.csv')